#  Discover a Superhero that Has the Most Appearances with other Superheroes

Marvel-Graph: HeroID, HeroesIDs_List;  a hero may span multiple lines

Marvel-Names: (ID, Name)

In [1]:
import findspark
findspark.init()

import pyspark
findspark.find()

'c:\\spark'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import collections

conf = pyspark.SparkConf().setAppName('SuperHero').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
def countCoOccurences(line):
    elements = line.split()
    return (int(elements[0]), len(elements) - 1)  ### (heroID, num_of_Co-occurences) per line

def parseNames(line):
    fields = line.split('\"')
    return (int(fields[0]), fields[1].encode("utf8"))

In [4]:
names = sc.textFile("Marvel-Names.txt")
namesRdd = names.map(parseNames)

lines = sc.textFile("Marvel-Graph.txt")

In [5]:
pairings = lines.map(countCoOccurences)   ### pairing per line
totalFriendsByCharacter = pairings.reduceByKey(lambda x, y : x + y)   ### aggregating lines by heroID
flipped = totalFriendsByCharacter.map(lambda xy : (xy[1], xy[0]))

In [6]:
mostPopular = flipped.max()

mostPopularName = namesRdd.lookup(mostPopular[1])[0]

print(str(mostPopularName) + " is the most popular superhero, with " + \
    str(mostPopular[0]) + " co-appearances.")

b'CAPTAIN AMERICA' is the most popular superhero, with 1933 co-appearances.
